In [4]:
pip install tensorflow-gpu 

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [44 lines of output]
  Traceback (most recent call last):
    File "D:\python2\Lib\site-packages\setuptools\_vendor\packaging\requirements.py", line 35, in __init__
      parsed = _parse_requirement(requirement_string)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "D:\python2\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 64, in parse_requirement
      return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "D:\python2\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 82, in _parse_requirement
      url, specifier, marker = _parse_requirement_details(tokenizer)
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "D:\python2\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 126, in _parse_require

In [2]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [3]:
df = pd.read_csv(os.path.join("jigsaw_comment_toxicity", "train.csv")) 

In [4]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
df.iloc[6]["comment_text"]

'COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK'

In [6]:
df[df.columns[2:]].iloc[6]

toxic            1
severe_toxic     1
obscene          1
threat           0
insult           1
identity_hate    0
Name: 6, dtype: int64

In [7]:
from tensorflow.keras.layers import TextVectorization

In [8]:
x = df["comment_text"]
y = df[df.columns[2:]].values

In [9]:
MAX_WORDS = 500000 #no. of words in vocab

In [10]:
vectorizer = TextVectorization(max_tokens = MAX_WORDS, output_sequence_length = 1800, output_mode = "int")

In [11]:
vectorizer.adapt(x.values)

In [22]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [24]:
vectorized_text = vectorizer(x.values)

In [25]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[   645,     76,      2, ...,      0,      0,      0],
       [219427,     54,   2489, ...,      0,      0,      0],
       [   425,    441,     70, ...,      0,      0,      0],
       ...,
       [ 32445,   7392,    383, ...,      0,      0,      0],
       [     5,     12,    534, ...,      0,      0,      0],
       [     5,      8,    130, ...,      0,      0,      0]], dtype=int64)>

In [26]:
# Tensorflow data pipeline
# M C SH BA P - map, chache, shuffle, batch, prefetch
#from_tensor_slices or list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) #helps prevent bottlenecks

In [30]:
batch_x, batch_y = dataset.as_numpy_iterator().next()

In [31]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [32]:
train_generator = train.as_numpy_iterator()

In [33]:
train_generator.next()

(array([[   70,     6,  3275, ...,     0,     0,     0],
        [  407,  1431,   331, ...,     0,     0,     0],
        [    8,    47,    69, ...,     0,     0,     0],
        ...,
        [ 4130,  8430,     5, ...,     0,     0,     0],
        [  264,    16,   319, ...,     0,     0,     0],
        [60902,    76,    51, ...,     0,     0,     0]], dtype=int64),
 array([[1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dropout, Dense, Embedding

In [40]:
model = Sequential()
#create embedding layer
model.add(Embedding(MAX_WORDS+1, 32))
#Bidirectional LSTM layer
model.add(Bidirectional(LSTM(32, activation = "tanh")))
#Feature extractor fully connected layer/Dense layer
model.add(Dense(128, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dense(128, activation = "relu"))
#Final Layer
model.add(Dense(6, activation = "sigmoid"))

In [42]:
model.compile(loss = "BinaryCrossentropy", optimizer = "Adam")

In [44]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(train, epochs = 5, validation_data=val)

Epoch 1/5
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 10452s 1s/step - loss: 0.0461 - val_loss: 0.0402
Epoch 2/5
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 6344s 908ms/step - loss: 0.0406 - val_loss: 0.0341
Epoch 3/5
2680/6981 ━━━━━━━━━━━━━━━━━━━━ 4:35:04 4s/step - loss: 0.0351

In [48]:
from matplotlib import pyplot as plt

In [50]:
history.history

{'loss': [0.06185799464583397], 'val_loss': [0.04560297727584839]}

In [52]:
input_text = vectorizer("You Freaking Suck! I am going to hurt you.")

In [56]:
batch = test.as_numpy_iterator().next()

In [66]:
res = model.predict(batch_x)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [54]:
model.predict(np.expand_dims(batch_x))

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


array([[0.98851573, 0.200391  , 0.88705415, 0.04613307, 0.7652535 ,
        0.15828751]], dtype=float32)

In [58]:
batch_x, batch_y = test.as_numpy_iterator().next()

In [60]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [62]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [68]:
for batch in test.as_numpy_iterator():
    x_true, y_true = batch
    y = model.predict(x_true)
    y_true = y_true.flatten()
    y = y.flatten()
    pre.update_state(y_true, y)
    re.update_state(y_true, y)
    acc.update_state(y_true, y)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [69]:
print(f"Precision {pre.result().numpy()}, Recall {re.result().numpy()}, Accuracy {acc.result().numpy()}")

Precision 0.8025435209274292, Recall 0.6916642785072327, Accuracy 0.4964894652366638


In [72]:
pip install gradio jinja2

Note: you may need to restart the kernel to use updated packages.Collecting gradio
   ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
   - -------------------------------------- 1.1/42.3 MB 34.9 MB/s eta 0:00:02
   - -------------------------------------- 1.6/42.3 MB 32.9 MB/s eta 0:00:02
   - -------------------------------------- 1.6/42.3 MB 32.9 MB/s eta 0:00:02
   - -------------------------------------- 1.6/42.3 MB 32.9 MB/s eta 0:00:02
   - -------------------------------------- 1.6/42.3 MB 32.9 MB/s eta 0:00:02
   - -------------------------------------- 1.6/42.3 MB 32.9 MB/s eta 0:00:02
   - -------------------------------------- 1.6/42.3 MB 5.8 MB/s eta 0:00:08
   -- ------------------------------------- 2.3/42.3 MB 7.2 MB/s eta 0:00:06
   -- ------------------------------------- 3.1/42.3 MB 8.4 MB/s eta 0:00:05
   --- ------------------------------------ 3.4/42.3 MB 8.3 MB/s eta 0:00:05
   --- ------------------------------------ 3.7/42.3 MB 8.4 MB/s eta 0:

In [74]:
import gradio as gd

In [ ]:
model.save("")